In [1]:
# Importing necessary models
import warnings
warnings.filterwarnings('ignore')

import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
import time
%matplotlib inline

import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from sqlalchemy import *
from sqlalchemy import create_engine

from functools import reduce

import tensorflow as tf

import sqlite3 as sql
pd.options.display.float_format = '{:,.4f}'.format

init_notebook_mode(connected=True)

import os

dbs_dir = 'C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Data\\DBs'

os.chdir(dbs_dir)

earn_engine = create_engine('sqlite:///earningsHistory.db', echo=False)
yahoo_engine = create_engine('sqlite:///yahoo.db', echo = False)
reuters_engine = create_engine('sqlite:///reuters.db', echo=False)

In [2]:
period_fundamental_tables = list(filter(lambda x: 'quarterly' in x, yahoo_engine.table_names()))

In [3]:
# 'SELECT * FROM overviews WHERE Underlying IN ' + str(tuple(curr_earnings.index))

fa_df = []

for yahoo_table in period_fundamental_tables:

    yahoo_engine_query = 'SELECT * FROM {0}'.format(yahoo_table)

    yahoo_data = pd.read_sql_query(yahoo_engine_query,
                                   con = yahoo_engine).drop_duplicates()
    yahoo_data.quarter = pd.to_datetime(yahoo_data.quarter)
    yahoo_data = yahoo_data.sort_values(['Underlying','quarter']).reset_index(drop = True)
    
    if len(fa_df) == 0:
        fa_df = yahoo_data
    else:
        fa_df = pd.merge(fa_df, yahoo_data,
                         how = 'inner', on = ['quarter','Underlying'])
        

keep_cols = list(filter(lambda x: '_y' not in x and 
                        'maxAge' not in x and 
                        'minorityInterest' not in x, fa_df.columns.tolist()))
fa_df = fa_df[keep_cols].fillna(np.nan).replace('na',np.nan)

fa_df = fa_df[fa_df.Underlying.isin(fa_df[['quarter',
                                           'Underlying']][fa_df[['quarter',
                                                                 'Underlying']].duplicated()].Underlying.drop_duplicates().tolist()) == False]

fa_df = fa_df.set_index(['quarter','Underlying']).apply(lambda x: pd.to_numeric(x)).reset_index()

earnings_returns = pd.read_sql_query('SELECT * FROM postEarningsReturns', 
                                     con = earn_engine).drop_duplicates().drop_duplicates()

earnings_returns.earningsDate = pd.to_datetime(earnings_returns.earningsDate)
earnings_returns = earnings_returns.sort_values(['Underlying','earningsDate']).reset_index(drop = True)

In [40]:
key_periods = fa_df[['Underlying','quarter']].groupby(['quarter']).count()
key_periods = key_periods[key_periods.Underlying > 3000]

key_names = fa_df[fa_df.quarter.isin(key_periods.index.tolist())].groupby(['Underlying']).count()
key_names = key_names[key_names.quarter == len(key_periods)].index.tolist()

pre_tensor = fa_df[fa_df.Underlying.isin(key_names) & 
                   fa_df.quarter.isin(key_periods.index.tolist())].sort_values(['quarter','Underlying']).reset_index(drop = True)

In [41]:
n_obs = len(pre_tensor.quarter.drop_duplicates())
n_rows = len(pre_tensor.Underlying.drop_duplicates())
n_features = len(pre_tensor.columns)

In [70]:
pre_tensor.values.reshape(n_features,n_obs,n_rows)[0][0]

array([Timestamp('2017-12-31 00:00:00'), 'AA', 1898000000.0, ..., 0.23,
       0.043, 0], dtype=object)

In [47]:
pre_tensor

,quarter,Underlying,accountsPayable,capitalSurplus,cash,commonStock,deferredLongTermAssetCharges,deferredLongTermLiab,goodWill,intangibleAssets,...,netIncomeFromContinuingOps,nonRecurring,operatingIncome,otherItems,otherOperatingExpenses,researchDevelopment,sellingGeneralAdministrative,totalOperatingExpenses,totalOtherIncomeExpenseNet,totalRevenue
0,2017-12-31,AA,"1,898,000,000.0000","9,590,000,000.0000","1,358,000,000.0000","2,000,000.0000","814,000,000.0000",nan,"154,000,000.0000","62,000,000.0000",...,"-56,000,000.0000",nan,"559,000,000.0000",nan,"-4,000,000.0000","9,000,000.0000","70,000,000.0000",2615000000,-343000000,3174000000
1,2017-12-31,AAL,"1,688,000,000.0000","5,714,000,000.0000","295,000,000.0000","5,000,000.0000","427,000,000.0000",nan,"4,091,000,000.0000","2,203,000,000.0000",...,"258,000,000.0000",nan,"970,000,000.0000",nan,"1,231,000,000.0000",nan,"383,000,000.0000",9631000000,-545000000,10601000000
2,2017-12-31,AAN,"80,821,000.0000","270,043,000.0000","51,037,000.0000","45,376,000.0000","11,589,000.0000",nan,"622,948,000.0000","279,151,000.0000",...,"177,560,000.0000",nan,"110,653,000.0000",nan,nan,nan,"327,745,000.0000",773974000,-49915000,884627000
3,2017-12-31,AAOI,"43,624,000.0000","285,376,000.0000","82,936,000.0000","19,000.0000","12,801,000.0000",nan,nan,"4,811,000.0000",...,"5,716,000.0000",nan,"10,320,000.0000",nan,nan,"10,670,000.0000","11,164,000.0000",69535000,-901000,79855000
4,2017-12-31,AAON,"10,967,000.0000",nan,"21,457,000.0000","210,000.0000",nan,nan,nan,nan,...,"15,770,000.0000",nan,"17,561,000.0000",nan,nan,nan,"13,291,000.0000",86599000,-111000,104160000
5,2017-12-31,AAT,"38,069,000.0000","919,066,000.0000","82,610,000.0000","473,000.0000",nan,"21,549,000.0000",nan,"9,776,000.0000",...,"9,731,000.0000",nan,"23,792,000.0000",nan,nan,nan,"6,860,000.0000",56701000,-13815000,80493000
6,2017-12-31,AAWW,"65,740,000.0000","715,735,000.0000","280,809,000.0000","301,000.0000","915,000.0000","236,904,000.0000","40,361,000.0000","66,124,000.0000",...,"209,454,000.0000",nan,"107,865,000.0000",nan,"45,522,000.0000",nan,"39,189,000.0000",520087000,-856000,627952000
7,2017-12-31,AAXN,"8,592,000.0000","201,672,000.0000","75,105,000.0000","1,000.0000","15,755,000.0000","6,803,000.0000","14,927,000.0000","18,823,000.0000",...,"-2,071,000.0000",nan,"8,648,000.0000",nan,nan,"14,755,000.0000","39,613,000.0000",86003000,-1582000,94651000
8,2017-12-31,AB,nan,nan,nan,"1,592,187,000.0000",nan,nan,nan,nan,...,"78,593,000.0000",nan,"85,725,000.0000",nan,nan,nan,nan,0,0,85725000
9,2017-12-31,ABB,"5,419,000,000.0000","29,000,000.0000","4,526,000,000.0000","188,000,000.0000","1,250,000,000.0000",nan,"11,199,000,000.0000","2,581,000,000.0000",...,"423,000,000.0000",nan,"933,000,000.0000",nan,"-106,000,000.0000","398,000,000.0000","1,543,000,000.0000",8347000000,-352000000,9280000000
